

Group nr: 48

Name 1 and CID: Othman Belal (CID) Othmanb

Name 2 and CID: Soheb Kanon (CID) Soheb

In [20]:
import numpy as np
import copy
import random
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from mining_world import Environment
from IPython.display import Image


# Introduction

This assignment dels with data structures, K-NN, tree-based classifiers and evaluation of the methods deployed on a fictional scenario.  

## Mining world 

<img src="imgs/poster.png" width="800"/>

## Scenario


Humanity has now reached a point where we need to extract and refine more Copium, a precious resource with great value. The only problem is that Copium can only be found on certain uninhabitable planets. This of course means that automated robots are sent instead.      

Copium is naturally very unstable and is only exists very temporary before it decays. There are very specific geological activities and circumstances needed for copium to form. The life cycle of Copium follows. First, a hot stream of liquid magma flows to the surface, creating a hotspot that that looks like a small crater. At the surface, if the conditions are correct, copium can form during the cool-down period. But as stated previously, Copium is unstable in its natural environment and decays to other materials shortly after. 

The formation of these deposits craters are very random, but the heat from them can easilly be detected with a satellite. But there is no way of knowing if the newly formed depoist contains copium from just a satellite, therefor there is a robot rover on the ground with sensors that can collect further measurements. The rover has many sensors that can measure the properties of the ground below it, but of course, Copium can not directly be detected with these types of sensors. This is where the machine learning approach will be used, to take all those measurements and try to classify if the deposit contains Copium or not.  


<img src="imgs/overView.png" width="500"/>


# Navigation - Tree search

This section will show how the naivigation is done. This is not a part of the assignment to understand, but will be used. 

##  Breadth first

The method used is a breadth first search algorithm, it is one of the simplest tree search algorithms and basically tries every option for a fixed number of steps and chooses the best one. The robot only have 4 actions, it can move at every time step in a chosen direction. 

## Actions 

<img src="imgs/actions.png" width="300"/>


In [21]:
class Node():
    def __init__(self, actor):
        self.actor = actor
        self.total_score = 0
    
    def update(self, action, inherited_score):
        score = self.actor.step(action)
        self.total_score = 1.05*inherited_score + score
        return self.total_score
    
    def get_score(self):
        return self.total_score

In [22]:
def breadth_first_search(actor, max_depth, action_space):
    node = Node(copy.deepcopy(actor)) 
    queue_keys = ['0'] # queue to keep track of nodes that has not yet been expanded.  
    visited = {queue_keys[0]: node} # saves visited nodes in order to not recalulate the entire path for each step. 
    
    max_score = -np.inf
    best_action = None

    while True:
        key = queue_keys.pop(0)
        if len(key) > max_depth: # stop at a set depth 
            break    
        node = visited[key]
        
        for action in action_space: # expand all children nodes
            child_node = copy.deepcopy(node)  # copy current node
            score = child_node.update(action=action, inherited_score=node.get_score()) # update node with action
            child_key = key + action # create child node key
            
            if score > max_score: # save best path 
                max_score = score
                best_action = child_key[1]
                
            visited[child_key] = child_node  # add child node to visited nodes.
            queue_keys.append(child_key)  # add child node queue of non expanded nodes. 
            
    return best_action


# Collect data

The first step is to collect data that will be used for both training and validation. The available types features can be seen with env.get_sensor_properties() and the actual measurements can be retrieved with env.get_sensor_readings(). It will return a dictionary with the same keys as in env.get_sensor_properties() containg a value for each feature. If the robot is not currently over a deposit, then it will return None. The true label can be extracted with env.get_ground_truth(), which will return a 1 if there is copium in the deposit and 0 if not. 
  

In [23]:
# Have True for first time running and then set to false. 
collect_data = False



In [24]:
env = Environment(map_type=1, fps=500, resolution=(1000, 1000))
env.exit()
sensor_properties = env.get_sensor_properties()
print('Sensor properties', sensor_properties)


Sensor properties ['ground_density', 'moist', 'reflectivity', 'silicon_rate', 'oxygen_rate', 'iron_rate', 'aluminium_rate', 'magnesium_rate', 'undetectable']


In [25]:
if collect_data == True:
    env = Environment(map_type=1, fps=500, resolution=(1000, 1000))
    sensor_properties = env.get_sensor_properties()

    # We can initilize the dictionary the following way.
    data = dict()
    data['copium'] = [] 
    for key in sensor_properties:
        data[key] = []

    for i in range(5000):
        action = breadth_first_search(actor=env.get_actor(), max_depth=3, action_space=env.get_action_space())
        env.step(action)
        # if we are over a deposit. 
        if env.get_sensor_readings() is not None:
            sensor_readings = env.get_sensor_readings()
            copium = env.get_ground_truth()

            for key in sensor_readings:
                data[key].append(sensor_readings[key])
            data['copium'].append(copium)

        env.render()

    env.exit()


# Exerscie 1: Data structure

## a) Pandas data frame 
In this assignment we will work pandas data frame for storing the collected data. First create a pandas data frame from the dictionary. The documentation for it can be found at https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html, only the data feild needs to be filled in with the created dictionary. Call this data frame df. 


In [26]:
# TODO: create pandas dataframe

if collect_data == True:
    df = pd.DataFrame(data)
    print(df)


## b) Save data
Collecting data for each time the notebook is opened is time consuming and the tests are not as reproducable if the data is different each time. Therefore we save the dateframe and we can load the data instead. After this you can set collect_data=False above. Use the function pandas function df.to_csv('name_of_file.csv', index=False)

The task is to save the data in a file named mydata.csv

In [27]:
if collect_data == True:
    # TODO: save data to a csv file named mydata.  
    df.to_csv('mydata.csv', index=False)
print(df)

      copium  ground_density     moist  reflectivity  silicon_rate  \
0          0        1.985926  0.072122      0.179557      0.135894   
1          0        1.039500  0.176045      0.597826      0.182387   
2          1        0.849743  0.289353      0.235944      0.227373   
3          0        1.098408  0.134335      0.295676      0.138176   
4          0        0.701561  0.100946      0.073247      0.123383   
...      ...             ...       ...           ...           ...   
1636       0        1.986117  0.087738      0.404132      0.118736   
1637       0        1.940350  0.269873      0.349480      0.055225   
1638       0        1.059966  0.121789      0.205741      0.128159   
1639       0        1.141077  0.008505      0.536344      0.059427   
1640       1        1.028524  0.283360      0.044964      0.055858   

      oxygen_rate  iron_rate  aluminium_rate  magnesium_rate  undetectable  
0        0.005734   0.478872        0.267396        0.100366      0.011739  
1    

## c) Load data
The task is to load the data from mydata.csv into a dataframe named df. Use the function pd.read_csv('name_of_file.csv')

In [28]:

# TODO: read data from mydata.csv into a dataframe called df.  
df = pd.read_csv('mydata.csv')
df

,copium,ground_density,moist,reflectivity,silicon_rate,oxygen_rate,iron_rate,aluminium_rate,magnesium_rate,undetectable
0,0,1.985926,0.072122,0.179557,0.135894,0.005734,0.478872,0.267396,0.100366,0.011739
1,0,1.039500,0.176045,0.597826,0.182387,0.008510,0.610552,0.079578,0.064772,0.054201
2,1,0.849743,0.289353,0.235944,0.227373,0.063420,0.593142,0.047785,0.034312,0.033968
3,0,1.098408,0.134335,0.295676,0.138176,0.029023,0.395689,0.398234,0.031218,0.007660
4,0,0.701561,0.100946,0.073247,0.123383,0.058315,0.297729,0.351371,0.150060,0.019143
...,...,...,...,...,...,...,...,...,...,...
1636,0,1.986117,0.087738,0.404132,0.118736,0.030829,0.342530,0.269732,0.168772,0.069401
1637,0,1.940350,0.269873,0.349480,0.055225,0.062243,0.418907,0.390706,0.032332,0.040587
1638,0,1.059966,0.121789,0.205741,0.128159,0.034269,0.243141,0.444174,0.080972,0.069284
1639,0,1.141077,0.008505,0.536344,0.059427,0.021310,0.700959,0.121238,0.020382,0.076684


In [29]:

all_features_without_copium = df.drop(columns='copium')
print("All features without copium \n", all_features_without_copium)

All features without copium 
       ground_density     moist  reflectivity  silicon_rate  oxygen_rate  \
0           1.985926  0.072122      0.179557      0.135894     0.005734   
1           1.039500  0.176045      0.597826      0.182387     0.008510   
2           0.849743  0.289353      0.235944      0.227373     0.063420   
3           1.098408  0.134335      0.295676      0.138176     0.029023   
4           0.701561  0.100946      0.073247      0.123383     0.058315   
...              ...       ...           ...           ...          ...   
1636        1.986117  0.087738      0.404132      0.118736     0.030829   
1637        1.940350  0.269873      0.349480      0.055225     0.062243   
1638        1.059966  0.121789      0.205741      0.128159     0.034269   
1639        1.141077  0.008505      0.536344      0.059427     0.021310   
1640        1.028524  0.283360      0.044964      0.055858     0.062739   

      iron_rate  aluminium_rate  magnesium_rate  undetectable  
0    

## d) Part 1: Split data

Here we will devide the data into a training set and a test set. Good rule of thumb is to use 80% of the data in the training set and 20 % in the test set. The the two data sets should be randomly sampled (shuffle=True), we also set he seed with random_state so that the split is reproducable. This is done with train_test_split() from sklearn, https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html. The syntax looks like 

train, test = train_test_split(dataframe, test_size=ratio_test_set, shuffle=True, random_state=42)

Why is it important that the data is shuffled when it is split, what could happen otherwise?

Text answer:
Shuffling data when you divide it into different parts is super important for making sure that computers learn things properly. It's like mixing up a deck of cards before playing a game so that no one gets an advantage.

If you don't shuffle, the computer might learn things in the wrong order, like memorizing a song but mixing up the lyrics. Shuffling helps the computer learn things better and be ready for different situations.

So, in simple terms, shuffling data is like mixing up the cards to help the computer learn and play the game (or solve problems) better.


In [30]:
# TODO: devide the data into train and test set. 
train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)


## e) Part 1: Analyse data balance

The occurance can be retrevied with .value_counts() from a pandas date frame. Here get the occurance of copium in the samples. Is the dataset balanced?

In [31]:
# TODO: Print number of samples with copium and the number of samples without copium.
df.value_counts('copium')


copium
0    1365
1     276
Name: count, dtype: int64

Text answer: No. In total there are 1611 samples wheras about 83.7% samples with no copium and 16.3% with copium


## e) Part 2: Balance data

We will throughout this assignment test each method with the original dataset and a balanced dataset. We show how balanceing can be done by downsampling below. Your task is to create an upsampled balanced dataset. You only need to use the upsampled data set for the rest of the other part 2) exerices. 

What could be the reason for chosing either upsampled or downsampeled metohd to balance the dataset?


Text answer:
Downsampling can save reminiscence and computation time, particularly when working with big datasets. However, it could result in loss of facts from the bulk leading to reduced our model's performance.

Upsampling can assist with increasing the accuracy of our model as it presents extra data points.

In [32]:
# TODO balance data det. 
# Seperate the data into something that contains copium and one that doesn't,
# can for example be done with df[df["copium"]==0] etc.

df_zero = df[df['copium']==0]
df_one = df[df['copium']==1]

train_zero = train[train["copium"]==0]
train_one = train[train["copium"]==1]


train_zero_downsampled = resample(train_zero,
                               n_samples=train_one.shape[0])

train_balanced_downsampled = pd.concat([train_one, train_zero_downsampled])


# TODO: upsample minority 
train_one_upsampled = resample(train_one,
                               n_samples=train_zero.shape[0])
train_balanced_upsampled = pd.concat([train_zero, train_one_upsampled])

# Exercise 3: Performance evaluation

Here we will define a class that later will be used for evaluation the performance of the classification methods. It will be used to log the accuracy, precsision and recalll. More information about precision and recall can be found at https://en.wikipedia.org/wiki/Precision_and_recall. 

Expliain why the different metics are usefull and why is not always accuarcy enough?

Text answer:
becasue the more info we get the more accurate we can search. For instance, calculating accuracy gives us the total of coorect guesess while callculating recall gives us the true positive in our data. Precision gives the result of our True amount of out posiive gueses. 



In [36]:
class Classification_eval(object):
    def __init__(self):
        # counters 
        self.TP = 0 # correctly identified positive 
        self.FP = 0 # falsely identified positive 
        self.TN = 0 # correctly identified negative 
        self.FN = 0 # falsely identified negative 
    
    def update(self, pred, label):
        """
        pred - is the prediction will be either a 1 or 0. 
        label - is the correct answer, will be either a 1 or 0.
        """
        # TODO: add to one of the counters each time this function is called. 
        if pred == 1 and label == 1:
            self.TP = self.TP + 1
 
        elif pred == 1 and label == 0:
            self.FP = self.FP + 1
        
        elif pred == 0 and label == 0:
            self.TN = self.TN + 1
        
        elif pred == 0 and label == 1:
            self.FN = self.FN + 1


    def accuracy(self): 
        # returns the accuracy 
        if (self.TP + self.TN) == 0:
            return 0
        # TODO: calculate the accuracy.
        
        accuracy = (self.TP + self.TN)/(self.TP + self.FP + self.TN + self.FN)
        return np.round(accuracy, 4)
    
    def precision(self): # percentage of the estimated positive that actually is positive
        if (self.TP + self.FP) == 0:
            return 0
        # TODO: calculate the precision.
        
        precision = self.TP/(self.TP + self.FP)
        return np.round(precision, 4)
    
    def recall(self): # percentage of correctly identified positive of the total positive
        if (self.TP + self.FN) == 0:
            return 0
        # TODO: calculate the recall.
        
        recall = self.TP/(self.TP + self.FN)
        return np.round(recall, 4)

In [37]:
# Test Classification_eval

test_eval = Classification_eval()
test_eval.update(1,1)
test_eval.update(1,1)
test_eval.update(1,1)
test_eval.update(0,1)
test_eval.update(1,0)
test_eval.update(1,0)
test_eval.update(0,0)
test_eval.update(0,0)
test_eval_ = np.allclose([test_eval.accuracy(), test_eval.precision(), test_eval.recall()], [0.625, 0.6, 0.75])
assert test_eval_
if test_eval_:
    print('Test successful')
else:
    print('Test failed')
    


Test successful


# Exercise 4: K- nearest neighbours

## a) Normalize

Here we will code our K-NN classifier, method 2.1 on page 21 in the book has the psudo code for K-NN. We will start with the data normalization, i.e. we will normalize the input data so that each feature has the same range in terms of max/min values. The min value can be found with data.min(), similarly for the max value. 

Why is it important that the data is normalized for the K-NN algorithm?

Text answer:
Normalization helps the algorithm make fair and accurate decisions by giving equal weight to all features, making it more robust and reliable in identifying the nearest neighbors and predicting outcomes.


In [38]:
class Normalize(object):
    def __init__(self):
        self.min = None
        self.max = None
    
    def normalize(self, data):
     
        return (data-self.min)/(self.max-self.min)
    
    def update_normalization(self, data):
        
        self.min = data.min()
        self.max = data.max()
        return self.normalize(data)

    

## b) K-NN
Lets make the k-Nearest Neighbors (k-NN) algorithm, fill in the TODO. 

The core idea behind k-NN is to make predictions based on the "neighborhood" of data points in a feature space. In this impementation we will use the $l^2$-norm for distance to determine the k nearest samples. The classification is done with a majoiry vote of the k nearest samples.

In [39]:

df.describe()

,copium,ground_density,moist,reflectivity,silicon_rate,oxygen_rate,iron_rate,aluminium_rate,magnesium_rate,undetectable
count,1641.000000,1641.000000,1641.000000,1641.000000,1641.000000,1641.000000,1641.000000,1641.000000,1641.000000,1641.000000
mean,0.168190,1.377081,0.153015,0.303482,0.135337,0.041576,0.408008,0.254732,0.110211,0.050137
std,0.374149,0.643805,0.086311,0.176327,0.078707,0.027344,0.160514,0.128155,0.066849,0.029805
min,0.000000,0.400000,0.000352,0.001800,0.011101,0.000005,0.045130,0.020363,0.006559,0.005487
25%,0.000000,0.901876,0.078777,0.149237,0.074648,0.020173,0.289588,0.152205,0.058777,0.026840
50%,0.000000,1.263650,0.154938,0.301963,0.128141,0.038002,0.432867,0.254191,0.102584,0.046565
75%,0.000000,1.759534,0.227704,0.458428,0.182192,0.057561,0.525580,0.342501,0.151056,0.065594
max,1.000000,4.591775,0.299972,0.599925,0.534004,0.158524,0.805996,0.668548,0.425536,0.248355


In [40]:
import numpy as np
from collections import Counter


class KNN(object):
    def __init__(self, k):
         self.features = None # normalized features from training data 
         self.labels = None # the corresponding labels (if there is copium)
         self.normalize = Normalize() # class instance for normalization
         self.k = k # the k value in k-nn algorithm.

    def fit(self, features, labels):
        
        self.features =self.normalize.update_normalization(features) #this was before like this features =self.Features
        self.labels = labels

    def predict(self, sample):
        # if len(sample) != len(self.features[0])
        #Because of this the test were giving me that error which says "index 569 is not .... "

        sample = self.normalize.normalize(sample)
        print(sample)
        distances = np.sqrt(np.sum((self.features - sample) ** 2, axis=1))
        nearest_indices = np.argsort(distances)[:self.k]
        nearest_labels = self.labels.iloc[nearest_indices]
        prediction = Counter(nearest_labels).most_common(1)[0][0]

        return prediction
    







In [41]:
# Test the KNN against known values. The test only covers small sample and there are hidden tests which will 
# tested after submitting. 

test_train = pd.read_csv('test.csv')
test_test = pd.read_csv('test2.csv')

train_labels = test_train['copium']
train_features = test_train.drop(columns='copium')
test_labels = test_test['copium']
test_features = test_test.drop(columns='copium')

knn = KNN(k=5)
knn.fit(train_features, train_labels)
pred_list = []

for i in range(test_features.shape[0]):
    pred = knn.predict(test_features.iloc[i])
    pred_list.append(pred)
val = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
if np.allclose(pred_list, val):
    print('Test successful')
else:
    print('Test failed')
    

    


ground_density    0.038913
moist             0.457343
reflectivity      0.182710
silicon_rate      0.232061
oxygen_rate       0.132379
iron_rate         0.820663
aluminium_rate    0.253579
magnesium_rate    0.354232
undetectable      0.487119
dtype: float64
ground_density    1.392249
moist             0.179695
reflectivity      0.600391
silicon_rate      0.126428
oxygen_rate       0.199055
iron_rate         0.897094
aluminium_rate    0.423164
magnesium_rate    0.123339
undetectable      0.165361
dtype: float64
ground_density    0.555765
moist             0.159419
reflectivity      0.910264
silicon_rate      0.345391
oxygen_rate       0.422837
iron_rate         0.485448
aluminium_rate    0.616728
magnesium_rate    0.256410
undetectable      0.342229
dtype: float64
ground_density    0.458618
moist             0.149286
reflectivity      0.245757
silicon_rate      0.166495
oxygen_rate       0.019104
iron_rate         0.839463
aluminium_rate    0.472716
magnesium_rate    0.084035
undetectab

## c) part 1: Evaluate the K-NN 
Evaluate the K-NN and choose a suitable k value. 

In [42]:

#train_labels = train['copium']
#train_features = train.drop(columns='copium')

train_labels = train_balanced_upsampled['copium']
train_features = train_balanced_upsampled.drop(columns='copium')

y = test['copium']
x = test.drop(columns='copium')

# TODO, try differenent values of k. 

knn = KNN(k=50)
knn.fit(train_features, train_labels)

log = Classification_eval()
for i in range(x.shape[0]):
    pred = knn.predict(x.iloc[i])
    log.update(pred, y.iloc[i])


print('Accuarcy', log.accuracy())
print('Precision', log.precision())
print('Recall', log.recall())


ground_density    0.135171
moist             0.370046
reflectivity      0.620552
silicon_rate      0.087878
oxygen_rate       0.438656
iron_rate         0.848867
aluminium_rate    0.025359
magnesium_rate    0.232098
undetectable      0.232897
dtype: float64
ground_density    0.243888
moist             0.469223
reflectivity      0.035783
silicon_rate      0.313570
oxygen_rate       0.323484
iron_rate         0.590829
aluminium_rate    0.174889
magnesium_rate    0.257161
undetectable      0.191104
dtype: float64
ground_density    0.147807
moist             0.523310
reflectivity      0.903999
silicon_rate      0.461955
oxygen_rate       0.026550
iron_rate         0.506329
aluminium_rate    0.079227
magnesium_rate    0.513272
undetectable      0.206551
dtype: float64
ground_density    0.000000
moist             0.868986
reflectivity      0.613350
silicon_rate      0.054999
oxygen_rate       0.091314
iron_rate         0.637913
aluminium_rate    0.496423
magnesium_rate    0.112316
undetectab

Try some differnet values of k and just looking at these resutlts would the klassifier work well for all k? 

Answer: Whеn k is sеt to lowеr valuеs, likе k=1 and k=5, thе classifiеr tеnds to prioritizе prеcision ovеr rеcall. In othеr words, it bеcomеs morе cautious and sеlеctivе in making positivе prеdictions. This can rеsult in a lowеr rеcall ratе, mеaning that it might miss somе positivе instancеs.

Whеn k is sеt to highеr valuеs, such as k=20 and k=50, thе classifiеr achiеvеs pеrfеct prеcision (1.0), but at thе cost of significantly rеducеd rеcall. In this scеnario, thе classifiеr bеcomеs ovеrly consеrvativе and lеans towards prеdicting thе majority class (nеgativе) most of thе timе, lеading to a lowеr rеcall for thе minority class.

Thе choicе of k dirеctly influеncеs thе balancе bеtwееn prеcision and rеcall, with lowеr k valuеs favoring prеcision and highеr k valuеs еmphasizing prеcision at thе еxpеnsе of rеcall.

| k | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 1 | 0.861 |0.861 |0.4 |  
| 5 | 0.8671| 0.65|0.26 |   
| 20 | 0.864|1.0 |0.1 |   
| 50 |0.861 |1.0 |0.08 |  


## c) part 2, balanced data. 
Now try with the upsampled balanced data, test for the same k values as in part 1. Are the results different and whitch one would identify more copium?  

Answer: The results table bellow shows that by using the upsampled balance data (UBD) in part 2 gives noticable higher recall values for K[1, 5, 20, 50] compared to the recall results in part 1. That leads to the conclosion that the classifier in part2 is better at identifying our positive instances, in our case the copium, in the UBD but it comes at a cost. If we look at the precision in part 2 it is getting lower values as we increase our k-value. This means thtat we might get more false positives, potentially more false identifying of copium. So it might be much better using the classifier in part 2 in case we prioritize identifying copium, but it is important to know that the chances of false positives increases. 

| k | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 1 | 0.861|0.5556 |0.4 |  
| 5 |0.8218|0.4416 | 0.68|   
| 20 | 0.8036|0.4194 |0.78 |   
| 50 |0.8308 | 0.4634| 0.76|

## d) kNN with Sklearn
Typically one would not implement these algorithms by hand for each time you need them. Here we show how the kNN algorithm from sklearn can be used. Note, it does not normalize the features by default. The advantanges with the sklearn version is in computational cost as they utilize clever tree structures to minimize the search problem. 

A quick note: Even if ready packages exist, it is still very valuable to understand the principels behind the algorithms. For example when the learning doesn't work as well as you liked (happens more often then not...), you can reason to why and modify the problem or method into something that does work.  

In [43]:
# Set up data used for fitting the model
train_labels = train['copium']
train_features = train.drop(columns='copium')

# with the off the shelf model we need normalize the data before 
normalize = Normalize()
data_features = normalize.update_normalization(train_features)

# The test data
y = test['copium']
x = test.drop(columns='copium')

# Normalize the test data
x_norm = normalize.normalize(x)

# Set up an knn with sklearn 
knn_sklearn = KNeighborsClassifier(n_neighbors=5)
knn_sklearn.fit(data_features.values, train_labels.values)

# Set up our own for comparison 
knn = KNN(k=5)
knn.fit(train_features, train_labels)

# Test both for the 20 first values
for i in range(20):
    pred = knn.predict(x.iloc[i])
    pred_sklearn = knn_sklearn.predict([x_norm.iloc[i]])
    print('Prediciton from our model', pred, ' and from sklearn ',pred_sklearn)


ground_density    0.135171
moist             0.370046
reflectivity      0.620552
silicon_rate      0.087878
oxygen_rate       0.438656
iron_rate         0.848867
aluminium_rate    0.025359
magnesium_rate    0.232098
undetectable      0.232897
dtype: float64
Prediciton from our model 0  and from sklearn  [0]
ground_density    0.243888
moist             0.469223
reflectivity      0.035783
silicon_rate      0.313570
oxygen_rate       0.323484
iron_rate         0.590829
aluminium_rate    0.174889
magnesium_rate    0.257161
undetectable      0.191104
dtype: float64
Prediciton from our model 0  and from sklearn  [0]
ground_density    0.147807
moist             0.523310
reflectivity      0.903999
silicon_rate      0.461955
oxygen_rate       0.026550
iron_rate         0.506329
aluminium_rate    0.079227
magnesium_rate    0.513272
undetectable      0.206551
dtype: float64
Prediciton from our model 0  and from sklearn  [0]
ground_density    0.000000
moist             0.868986
reflectivity      0

# Exercise 5: Tree based classifier

In this exercise we will implement a thee based classifier. The implementation will be a class representing a node and will recursivly create child node until the entire tree structure is built. The general step for learning a tree structure from learning data is as follows:

1. Initiate the the root node.
2. Call the learn function with the learning data. 

Inside the learn function

3. Check if all the data has the same classification, and check if the data set is smaller then min_node_size. If any of these are true, set the node as a leaf node and return. 
4. Find the spliting point in the data that returns the highest gini value, be careful to not use the label which we want to classify. 
5. Split the dataset, one is usually called head and the other tail. Save the splitting parameter and which value was used for the splitting. 
6. Check the prediction of the splited data sets (majority copium or not) and create two child nodes, one for each of the splited dataset. 
7. Call the the learn function of the child node with the corresponding split data and appdend the child node. After that return. 

The above will build the tree structure. To use the tree for prediction simply call the predict function of the root node with a data sample. In the predict function the following steps should happen.

1. Check if the current node is a lead node, if it is simply return the self.prediction of that node. 
2. If it is not a leaf node then check the value of the splitting parameter in the data sample and compare with the splitting value. 
3. Call the predict function of the correspoinding child node with the data sample and return the answer.

This will recurivly traverse the tree until a leaf node containing the prediction is reached. 

 


## 5 a) Find split point

First we define a function that can find the splitting point for a parameter with the highest gini value. 

The gini value can be described as:

If $\Gamma$ contains the set of all labeles, then $\Gamma(x < c)$ would be all labels that belong to the criteria $x < c$ , where $x$ is a feature or parameter and $c$ is the splitting criteia (split_value).  


For example: 

If $x$ is iron_rate and our splitting critera is $c = 0.4$, then $\Gamma(x < c)$ would be a set of all the label from samples with $\textrm{iron_rate} < 0.4$.


Then we can define:

$v_1 = mean(\Gamma(x < c))$

$v_2 = mean(\Gamma(x \geq c))$

$s_1 = v_1^2 + (1-v_1)^2$

$s_2 = v_2^2 + (1-v_2)^2$

We define $len(\Gamma)$ to give the number of samples in the dataset, then the weighted gini value is:

$s = \frac{len(\Gamma(x_i < c))}{len(\Gamma)}*s_1 + \frac{len(\Gamma(x_i \geq c))}{len(\Gamma)}*s_2$

The goal is to find a criteria $c$ that maximizes the gini value $s$. The easies is to order the data set after parameter, e.g. we could order all sample depending on how much iron_rate they have, then we can try to split the data between every data point in a for loop and save the one that has the highest gini value. The critieria c would be the average between two data points that are sorted, i.e. $c_i = \frac{x_{i-1} + x_i}{2}$. 

In [44]:
def find_split_point(data, label, parameter):
    sorted_data = data.sort_values(by=parameter)
    sorted_label = sorted_data[label]

    best_gini_value = 0.0
    best_split_value = None
    best_df_head = None
    best_df_tail = None

    for split_index in range(1, len(sorted_data)):
        # Calculate v1 and v2
        v1 = sorted_label[:split_index].mean()
        v2 = sorted_label[split_index:].mean()

        # Calculate s1 and s2
        s1 = v1**2 + (1 - v1)**2
        s2 = v2**2 + (1 - v2)**2

        # Calculate weighted gini value (s)
        len_head = split_index
        len_tail = len(sorted_data) - split_index
        s = (len_head / len(sorted_data)) * s1 + (len_tail / len(sorted_data)) * s2

        if s > best_gini_value:
            best_gini_value = s
            best_split_value = (sorted_data.iloc[split_index - 1][parameter] + sorted_data.iloc[split_index][parameter]) / 2
            best_df_head = sorted_data.head(split_index)
            best_df_tail = sorted_data.tail(len(sorted_data) - split_index)

    return best_split_value, best_gini_value, best_df_head, best_df_tail






# def find_split_point(data, label, parameter):
#     """
#     data - all the data we want to split, our (gamma)
#     label - the parameter we want to classify. 
#     parameter - the parameter we want to check for, our x_i
#     -----------
#     retrun:
#     split_value - the spliting value, our c. 
#     gini_value - the gini value for the best c.
#     df_head - the data frame belonging to x_i < c
#     df_tail - the data frame belonging to x_i => c
#     """
#     # Beging by sorting the data after the paramter. 
#     sorted_data = data.sort_values(by=parameter)
#     sorted_label = sorted_data[label]
    
    
#     # TODO loop through all the split points in the sorted data and find 
#     # the best gini_value (s) and split_value (c).  
    
#     # gini_value =         
#     # split_value =
    
#     # TODO: get the two data frames the corresponds to the split data. 
#     # the functions .head(split_index) and .tail(len(x) - split_index) could be useful. 
#     # df_head = 
#     # df_tail =   
    
#     return split_value, gini_value, df_head, df_tail


  





In [45]:
# Test for find_split_point function, the test does not test everything and more hidden tests are used 
# during grading. 
df_test = pd.read_csv('test.csv')
split_value, gini_value, df_head, df_tail = find_split_point(df_test, 'copium', 'ground_density')
if split_value == 1.8762144694722296 and gini_value == 0.7234487734487733:
    print('Test successful')
else:
    print('Test failed')
print(split_value)
print(gini_value)
    

Test successful
1.8762144694722296
0.7234487734487733


## 5 b) Tree Node

Here we will code a tree node class that can recursivly call it self. 


In [46]:
class TreeNode():
    def __init__(self, prediction=None):
        self.split_value = None # the splitting value (c)
        self.split_parameter = None # what feature where uesd for the split (x_i)
        self.child_nodes = [] # list that contains two child nodes, if not leaf_node
        self.leaf_node = 0 # is this leaf_node (0=no, 1=yes)
        self.prediction = prediction # classification made in this node.
        
    def predict(self, data):
        # TODO: To make a prediction we need to traverse the tree recursivly down to a leaf node and return 
        # the prediction. 
        # step 1: check if this is a leaf node, if it is then return prediction of this node, 
        # otherwise contine with step 2.
        # step 2: Compare the input data with the splitting criteria.
        # step 3: Call the prediction function in the corresponding child_node and return the prediction. 
        if self.leaf_node == True:
            return self.prediction
        if self.leaf_node == False:
            if data[self.split_parameter] < self.split_value: 
                return self.child_nodes[0].predict(data)
            else:
                return self.child_nodes[1].predict(data)
        
            
    # def learn(self, data, label, min_node_size):
    #     """
    #     data - the training data
    #     label - the parameter we want to classify, i.e. "copium"
    #     min_node_size - number of data points in a node for it to become a leaf node. 
    #     """
    #     # TODO: wirte the learning function. 

        # Step 1: check if this node should be turned into a leaf node, this happens if the data set is smaller 
        # then the min_node_size or if the data labels are homogenious. 
        
            
        # Step 2: Loop over all features and get the best gini and split_value for each feature. 
        # Save the best split value and split_paramter and the two new data frames 
        # corresponding to the split [df_head, df_tail] (these will be data frames for the child nodes).  
        
        
        # Step 3: Calculate the majority vote prediction of the two data frames. 
        # Create two child nodes, (e.g. child = TreeNode(prediction=1) for prediction copium) and and call the 
        # learn function with the corresponding dataframe. 
        
        # Step 4: append the the child node to the self.child_nodes. It should be in the order 
        # of the child node correspoinding to [df_head, df_tail].
        
    def learn(self, data, label, min_node_size):
        feature_names = data.columns.tolist()

        if (len(data) < min_node_size) or (np.mean(data[label]) == 1) or (np.mean(data[label]) == 0): 
            # If the data set is smaller than min_node_size or labels are homogeneous, make it a leaf node.
            self.leaf_node = True
            # Store the majority vote prediction in leaf nodes
            # self.prediction = df[label].mode().iloc[0]
            if np.mean(data[label]) <= 0.5:
                self.prediction = 0
            else:
                self.prediction = 1
        else:
            self.leaf_node = False

            best_split_value = 0
            best_gini_value = 0
            best_df_head = None
            best_df_tail = None

            for feature in feature_names:
                if feature == label:
                    continue
                split_value, gini_value, df_head, df_tail= find_split_point(data, label, feature)
                if df_head is not None and df_tail is not None:
                    if gini_value > best_gini_value:
                        
                        best_split_value, best_gini_value, best_df_head, best_df_tail, self.split_parameter = split_value, gini_value, df_head, df_tail,feature

            # Now you can store the best split information in the TreeNode object
            self.split_value = best_split_value
         

            if best_df_head is not None and best_df_tail is not None:
                # Create child nodes and call learn recursively for each child node
                self.child_nodes = [TreeNode(), TreeNode()]
                self.child_nodes[0].learn(best_df_head, label, min_node_size)
                self.child_nodes[1].learn(best_df_tail, label, min_node_size)

            return 
        

      

In [47]:
# Test to give indicaion if the TreeNode class is correct.

df_test = pd.read_csv('test.csv')
tree = TreeNode() # create root node
# learn the tree structure
tree.learn(df_test, "copium", min_node_size=4)

pred_list = []
pred_answer = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
               0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 
               0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
               1, 0, 1, 0, 1, 0, 0, 0, 0]

for i in range(df_test.shape[0]):
    pred = tree.predict(df_test.iloc[i])
    pred_list.append(pred)

print(pred_list)
print(pred_answer)



if np.allclose(pred_list, pred_answer):
    print('Test successful')
else:
    print('Test failed')


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
Test successful


##  Train the Tree Classifier

In [71]:
tree = TreeNode() 
#tree.learn(train, "copium", min_node_size=50)
tree.learn(train_balanced_upsampled, 'copium', min_node_size=1)


## Test Classifier

In [72]:
y = test['copium']
x = test.drop(columns='copium')

log = Classification_eval()

for i in range(x.shape[0]):
    pred = tree.predict(x.iloc[i])
    log.update(pred, y.iloc[i])
        
print('accuarcy', log.accuracy())
print('precision', log.precision())
print('recall', log.recall())

accuarcy 0.8815
precision 0.7
recall 0.5932


## 5 c) part 1

Use the non balanced data to learn the tree classifier and test the classifier for some different min_node_size values.

Answer:



| min_node_size | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 1  | 0.8754 | 0.65 | 0.661 |  
| 10 | 0.8723 | 0.6604 | 0.5932 |   
| 20 | 0.8693 | 0.6379 | 0.6271 |   
| 50 | 0.8723 | 0.7179 | 0.4746 |  

## 5 c) part 2, Try with balanced data

Use the balanced data to learn the tree classifier and test the classifier for some different min_node_size values.

Answer:


| min_node_size | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 1  | 0.8815 | 0.7 | 0.5932 |  
| 10 | 0.8845 | 0.6981 | 0.6271 |   
| 20 |0.8845  |0.6615  |0.7288  |   
| 50 | 0.8906 | 0.662  | 0.7966 |

## d) Tree classifier with sklearn 

Here we compare our tree classifier to the one implemented in sklearn. It is important to note that there can be differences between them, unlike our implementation the sklearn has some randomness in the traing and does not produce the same results each time. But overall they give simlar results. 

In [73]:
# Set up data used for fitting the model
train_labels = train['copium']
train_features = train.drop(columns='copium')

# Decition tree or classifier from sklearn 
tree_sklearn = DecisionTreeClassifier(min_samples_split=1.0, splitter='best', 
                             criterion='gini', min_samples_leaf=1)
# Train the sklearn tree classifer
tree_sklearn.fit(train_features.values, train_labels.values)

# Train our tree classifier 
tree = TreeNode() # create root node
tree.learn(train, "copium", min_node_size=1)

# Test both and compare
y = test['copium']
x = test.drop(columns='copium')
# classify the data
for i in range(30):
    pred_sklearn = tree_sklearn.predict([x.iloc[i]])
    pred = tree.predict(x.iloc[i])

    print('sklearn pred ',pred_sklearn, 'our tree pred ', pred)

sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [1] our tree pred  1
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  1
sklearn pred  [0] our tree pred  1
sklearn pred  [0] our tree pred  1
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [1] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] our tree pred  0
sklearn pred  [0] ou

## Exersice 6: Balance data analysis

1. It there a difference between that results of the balanced and non-balanced data for the tree classifier?
2. Is there a difference between the K-NN vs tree classifier?
3. Which one would you expect to be a faster classifier?

Answer:
1 According to both tables for balanced and non-balanced data, we observe that the accuracy, precision, and recall metrics are consistently better for the balanced tree compared to the non-balanced data across various node sizes.


2 Yes, there are many differences between K-NN and tree classifiers, from the way we implement them to their time and space complexity, and each of them has its own unique properties that affect the complexity of the code.

When it comes to K-NN, the code is relatively straightforward. We simply place the testing point in an n-dimensional space and calculate the Euclidean distance from every training data point around it. The testing point is then classified based on the majority of the nearest K values.

On the other hand, tree classifiers have specific properties, including a special case on the graph known as a binary tree. There are several other differences that have been discussed in the 'Tree-based Classifier.pdf' document available on Canvas under the 'Files' section.

3 When it comes to the time complexity of classifiers, the Decision Tree classifier exhibits a logarithmic time complexity of O(log n). This is primarily due to the fact that during classification, the test data is split at each node as we traverse down the tree, which results in a logarithmic time complexity. On the other hand, the k-Nearest Neighbors (K-NN) classifier operates differently. Its time complexity is O(k * n), where 'k' represents the number of neighbors considered, and 'n' is the size of the training dataset. In K-NN, the test data needs to iterate over the training data multiple times, making the time complexity linear in the size of the training dataset, i.e., O(n)



## Exersice 7: Deployment (Optional)

Here we will try the learned classifiers on a larger map. Make sure that the last run version of K-NN and tree have good parameters i.e. k and min_node_size values. Which one found more copium?


In [ ]:
env = Environment(map_type=2, fps=5, resolution=(1000, 1000))

try:
    sensor_properties = env.get_sensor_properties()
    sensor_sample = dict()
    for key in sensor_properties:
        sensor_sample[key] = [0]

    log_knn = Classification_eval()
    log_tree = Classification_eval()


    for i in range(500):
        action = breadth_first_search(actor=env.get_actor(), max_depth=3, action_space=env.get_action_space())
        env.step(action)
        if env.get_sensor_readings() is not None:
            sensor_readings = env.get_sensor_readings()
            for key in sensor_readings:
                sensor_sample[key][0] = sensor_readings[key]
            sensor_sample_df = pd.DataFrame(sensor_sample)
            log_knn.update(knn.predict(sensor_sample_df.iloc[0]), env.get_ground_truth())
            log_tree.update(tree.predict(sensor_sample_df.iloc[0]), env.get_ground_truth())
            env.plt_acc.update_acc(log_tree.accuracy(), log_knn.accuracy())
        env.render()

    env.exit()

    print("K-NN accuracy ", log_knn.accuracy(), "Tree accuracy", log_tree.accuracy())
    print("Number of copium deposits found, K-NN:", log_knn.TP, " Tree:", log_tree.TP)
except:
    env.exit()



